In [55]:
import pandas as pd
import openpyxl
import warnings

warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)

data_kenshin = pd.read_csv("./data/kenshin_mac_ctl.csv")
data_syobyo = pd.read_csv("./data/syobyo_mac_ctl.csv")
data_kanja = pd.read_csv("./data/kanja_mac_ctl.csv")

pID_list = list(set(data_kenshin["加入者id"]))

data_syobyo = data_syobyo[data_syobyo['疑いフラグ'].isna()]
data_kenshin["yearmonth"] = [str(x)[0:6] for x in data_kenshin["健診実施年月日"]]

syobyo_list = pd.read_excel("./data/disease_use_criteria_add_mac.xlsx")
use_disease = syobyo_list.loc[syobyo_list["use"]==1,:]


is_acute = {}

for i in range(len(use_disease.index)):
    # print(use_disease["病名"].values[i])
    is_acute[use_disease["病名"].values[i]] = use_disease["acute"].values[i]


data_kenshin_merged = pd.DataFrame()

for p in pID_list:
    data_kenshin_ind = data_kenshin.loc[data_kenshin["加入者id"]==p,:]
    data_kenshin_ind = data_kenshin_ind.sort_values('健診実施年月日')
    data_syobyo_ind = data_syobyo.loc[data_syobyo["加入者id"]==p,:]

    
    for d in use_disease["病名"]:
      
        if d in data_syobyo_ind["標準病名"].values:
            tmp_vec = []
            data_syobyo_ind_dis = data_syobyo_ind.loc[data_syobyo_ind["標準病名"]==d,:]
            for i in range(len(data_kenshin_ind.index)):
                ym_kenshin = data_kenshin_ind["yearmonth"].values[i]
                flag = 0
                for ym in data_syobyo_ind_dis["診療年月"]:
                    diff = int(ym_kenshin[0:4])*12 + int(ym_kenshin[4:6]) -int(str(ym)[0:4])*12 - int(str(ym)[4:6])

                    if is_acute[d] == 1:
                        if diff >=0 and diff <= 6:
                            flag = 1
                    else:
                        if diff >=0:
                            flag = 1
                tmp_vec.append(flag)

        else:
            tmp_vec = [0] * len(data_kenshin_ind.index)
        data_kenshin_ind[d] = tmp_vec
    data_kenshin_merged = pd.concat([data_kenshin_merged,data_kenshin_ind])

# 患者データのis_macを結合
data_kanja = pd.read_csv("./data/kanja_mac_ctl.csv")
data_kenshin_merged = pd.merge(data_kenshin_merged,data_kanja[['加入者id','is_mac']],on='加入者id')

data_kenshin_merged.to_csv("./data/Kenshin_Syobyo_merged.csv",index=False)
